In [1]:
import torch

# storageと和解せよ

![](https://m.media-amazon.com/images/I/51ZQBpy4-+L._AC_SX466_.jpg)

`Tensor`の多次元配列は`storage`に一次元配列として格納されていて、`Tensor`がもつメタ情報と合わせることで多次元配列として表現されている。

In [2]:
points = torch.tensor([[1, 2], [3, 4], [5, 6]], dtype=torch.float)
points2 = points.transpose(0, 1)
print('points storage', points.storage())
print('points2 storage', points2.storage())
points.storage is points2.storage

points storage  1.0
 2.0
 3.0
 4.0
 5.0
 6.0
[torch.FloatStorage of size 6]
points2 storage  1.0
 2.0
 3.0
 4.0
 5.0
 6.0
[torch.FloatStorage of size 6]


False

こんな感じでrow-majorである

In [3]:
points.zero_()
print('points storage', points.storage())
print('points2 storage', points2.storage())

points storage  0.0
 0.0
 0.0
 0.0
 0.0
 0.0
[torch.FloatStorage of size 6]
points2 storage  0.0
 0.0
 0.0
 0.0
 0.0
 0.0
[torch.FloatStorage of size 6]


In [4]:
points2

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [5]:
print('ptr points storage', points.storage().data_ptr())
print('ptr points2 storage', points2.storage().data_ptr())

ptr points storage 140660538230400
ptr points2 storage 140660538230400


`data_ptr()`を見れば同じ`storage`とわかる。

ここからは`storage`からいい感じに多次元配列を復元する過程を調べる。

`Tensor`が持ってるメタ情報は

- shape
- offset
- stride

In [7]:
points = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]], dtype=torch.float)

In [8]:
def meta_info(x):
    print('shape', x.shape)
    print('offset', x.storage_offset())
    print('stride', x.stride())

In [9]:
points2 = points[1:3, 1:3]
points2

tensor([[4., 5.],
        [7., 8.]])

In [10]:
meta_info(points)
print('=========')
meta_info(points2)

shape torch.Size([3, 3])
offset 0
stride (3, 1)
shape torch.Size([2, 2])
offset 4
stride (3, 1)


In [11]:
points.storage().data_ptr() == points2.storage().data_ptr()

True

In [12]:
points.storage()

 0.0
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
 7.0
 8.0
[torch.FloatStorage of size 9]

こんな感じでスライスだと同じstorageを持っていて、かつメタ情報から復元できる。疑似的に書いてみる.

In [13]:
storage = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
shape = (2, 2)
offset = 4
stride = (3, 1)

In [14]:
import numpy as np

In [15]:
tensor = np.empty(shape)

for i in range(shape[0]):
    for j in range(shape[1]):
        tensor[i, j] = storage[offset + i * stride[0] + j * stride[1]]

In [16]:
tensor

array([[4., 5.],
       [7., 8.]])

要は

$ a_0, a_1, ... a_n $　にアクセスするときは$i$次元目のstrideを $s_i$, offsetを$o$として

`storage`の $ o + \Sigma_i a_i * s_i $を見れば良い。

ちなみに今回は(`storage`の中身とインデックスを同じにしていたので) `storage`の4, 5, 7, 8番目にアクセスしているが、メモリアクセス的に嬉しくない。

これは `.is_contiguous()`で確かめられる

In [17]:
print('points is contiguous', points.is_contiguous())
print('points2 is contiguous', points2.is_contiguous())

points is contiguous True
points2 is contiguous False


連続したメモリ領域になるようにするには`contiguous()`を使えば良い

In [18]:
points3 = points2.contiguous()

In [19]:
print('points2 storage', points2.storage())
print('points3 storage', points3.storage())

points2 storage  0.0
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
 7.0
 8.0
[torch.FloatStorage of size 9]
points3 storage  4.0
 5.0
 7.0
 8.0
[torch.FloatStorage of size 4]


新しく`storage`を確保したのでメモリは消費するが、何回もアクセスするなら連続になるように新しく`storage`を確保する方がパフォーマンス的には嬉しそう。

TensorがCPU上にある時、`np.ndarray`との相互変換ではコピーが発生しない。(同じstorageを参照する)

In [20]:
points = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]], dtype=torch.float)
points.storage().data_ptr()

140660689943680

In [21]:
points = points.numpy()
points.__array_interface__

{'data': (140660689943680, False),
 'strides': None,
 'descr': [('', '<f4')],
 'typestr': '<f4',
 'shape': (3, 3),
 'version': 3}

In [22]:
points = torch.from_numpy(points)
points

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

In [23]:
print('points storage ptr', points.storage().data_ptr())

points storage ptr 140660689943680
